In [8]:
from DB_and_Azure import sql_db_functions as SQLf

In [11]:
conn, cursor = SQLf.connect_sql()

In [12]:
SQLf.truncate_all_tables(conn=conn,cursor=cursor,table='Products')

In [14]:
conn, cursor = SQLf.connect_sql()

query = f"SELECT * From Products;"
cursor.execute(query)

# Fetch the rows
rows = cursor.fetchall()
rows

[]

In [ ]:
#import streamlit as st
import pandas as pd

In [ ]:
from Features import world_constructions as WCf

In [ ]:
import base64
from io import BytesIO
from PIL import Image


def base64_to_image(base64_string):
    # Remove the data URI prefix if present
    if "data:image" in base64_string:
        base64_string = base64_string.split(",")[1]
    # Decode the Base64 string into bytes
    image_bytes = base64.b64decode(base64_string)
    return image_bytes

def create_image_from_bytes(image_bytes):
    # Create a BytesIO object to handle the image data
    image_stream = BytesIO(image_bytes)

    # Open the image using Pillow (PIL)
    image = Image.open(image_stream)
    return image

def resize_image(image, new_width):
    # Open the image

    # Calculate the new height while maintaining the aspect ratio
    original_width, original_height = image.size
    aspect_ratio = original_height / original_width
    new_height = int(new_width * aspect_ratio)

    # Resize the image
    resized_image = image.resize((new_width, new_height))

    return resized_image

### Loading Postgress

In [ ]:
df = WCf.world_construction.init_retail_gallery()
df.head(2)

In [ ]:
df[df.Brand == 'HM']

In [ ]:
i = 2
description = df.iloc[i].Summary
row = df.iloc[i]
detail = df.iloc[i].Detail

df.iloc[i]

In [ ]:
row['base64'][-len(row['base64'])+1]

In [ ]:
description

In [ ]:
resize_image(
    create_image_from_bytes(image_bytes=base64_to_image( row['base64'][1]   )),
    400)

In [ ]:
resize_image(
    create_image_from_bytes(image_bytes=base64_to_image( row['base64'][0]   )),
    400)

In [ ]:
df_retail = WCf.world_construction.init_retail_gallery()
df_retail.head()

### Loading Chroma

In [ ]:
vectorestore = WCf.world_construction.init_chroma_db()

In [ ]:
query = "Summer"
docs = vectorestore.similarity_search(query)

In [ ]:
docs[0].page_content

### Testing feature 1 from scratch

In [ ]:
from Features import search_from_luxury_brands as Sf

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_core.documents import Document


In [ ]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

In [ ]:
embedding_function = SentenceTransformerEmbeddings(model_name='all-mpnet-base-v2')

In [ ]:
store = InMemoryStore()
id_key = 'doc_id'

In [ ]:
from langchain_community.vectorstores import Chroma

In [ ]:
retriever = MultiVectorRetriever(
    vectorstore = vectorestore,
    byte_store = store,
    id_key=id_key, 
    search_kwargs={"k": 5},
    embedding_function = embedding_function
)

In [ ]:
doc_ids = range(14,57,1)

In [ ]:
df.iloc[1].Summary

In [ ]:
summary_docs = [
    Document(page_content=df.iloc[1].Summary, metadata={id_key: i})
    for i in range(14,57,1)
]

In [ ]:
retriever.docstore.mset(list(zip(doc_ids, summary_docs)))

In [ ]:
retriever.invoke('Summer')

In [ ]:
vectorstore.similarity_search('Summer')

## Feature 1: Find similar pieces from luxury brands in retail brands 

### library

In [ ]:
from Features import search_from_luxury_brands as Sf

In [ ]:
entity = Sf.seacrh_from_luxury_brands(description=description, vectorstore=vectorestore)

In [ ]:
answer = entity.search_similarity_from_description()

### Organize in daframe for display

In [ ]:
answer['answer']

In [ ]:
answer['answer'].replace('Piece_1: ','').replace('Piece_2: ','').replace('Piece_3: ','').replace('Piece_4: ','').replace('Piece_5: ','').replace('\n\n','').split('*')[1:]

In [ ]:
for i in range(0,4):
    print(answer['context'][i].page_content)

In [ ]:
for i in range(0,4):
    print(answer['context'][i].metadata['doc_id'])

In [ ]:
def organize_data(answer):

    page_content = []
    metadata_1 = []
    metadata_2 = []
    doc_id =[]

    answers = answer['answer'].replace('Piece_1: ','').replace('Piece_2: ','').replace('Piece_3: ','').replace('Piece_4: ','').replace('Piece_5: ','').replace('\n\n','').split('*')[1:]

    for i in range(0,4):

        doc_id.append(answer['context'][i].metadata['doc_id'])
        page_content.append(answer['context'][i].page_content)
        metadata_1.append(answer['context'][i].metadata['img_1'])
        metadata_2.append(answer['context'][i].metadata['img_2'])

    df_init = pd.DataFrame([doc_id,answers, page_content,metadata_1,metadata_2]).T

    df_init = pd.DataFrame(df_init)

    df_init.rename(columns={0:"Id",1:"Answer",2:"Summary",3:"Img_1",4:"Img_2"},inplace=True)


    return df_init

    

In [ ]:
df_recommendations = organize_data(answer=answer)
df_recommendations

In [ ]:
for index, row in df_recommendations.iterrows():
    print(row['Id'])

In [ ]:
df_retail[df_retail.Brand_id == 38]

### Recreating features

In [ ]:
### lets set up the system
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

from langchain_openai import ChatOpenAI


import sys, os 

apikey = 'sk-proj-OvVavmDwvsvUHryza7P7T3BlbkFJ9K11gPvYgUYrNbDFjUOd'

In [ ]:
#Setting up retriever
retriever = vectorestore.as_retriever(search_kwargs={"k": 5})

In [ ]:
#Getting a generalization of the description
description_generalization = ''

In [ ]:
os.environ['OPENAI_API_KEY'] = apikey

turbo_llm = ChatOpenAI(
    temperature=0.2,
    model_name='gpt-4o-mini'
)


In [ ]:
system_prompt = ( """
    
    You are a search engine for clothing. 
    Use the following retrieved context to find clothing with a similar descriptions.
    
    \n\n
    Context: "{context}"
    
    If you don't know the answer, say that you.
    """                         
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(turbo_llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

query = f"""
    I have this peace of clothing: {description_generalization}

    Answer ONLY THE FOLLOWING FORM, describing how the 5 pieces of clothing are similar to the one i have:
    *Piece_1: Explanation
    *Piece_2: Explanation
    *Piece_3: Explanation
    *Piece_4: Explanation
    *Piece_5: Explanation
    
"""

response = rag_chain.invoke({"input": description_generalization})


## Feature 2: Find pieces from user prompt description ( Text prompt to suggestions)

### Feature 3: Find pieces similar to user provided piece ( Image to suggestions)